# Why is d_dw sparse? and do I need twice the filters?

In [1]:
import numpy as np
import torch
import random

import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#from ..easy_ntk import calculate_NTK
from einops import rearrange

import time

#import sys
#from pathlib import Path

from numba import njit

#from numba import njit
#from numba.typed import List

from numba import njit

%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
import copy

def _del_nested_attr(obj, names):
    """
    Deletes the attribute specified by the given list of names.
    For example, to delete the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'])
    """
    if len(names) == 1:
        delattr(obj, names[0])
    else:
        _del_nested_attr(getattr(obj, names[0]), names[1:])

def _set_nested_attr(obj, names, value):
    """
    Set the attribute specified by the given list of names to value.
    For example, to set the attribute obj.conv.weight,
    use _del_nested_attr(obj, ['conv', 'weight'], value)
    """
    if len(names) == 1:
        setattr(obj, names[0], value)
    else:
        _set_nested_attr(getattr(obj, names[0]), names[1:], value)

def extract_weights(mod):
    """
    This function removes all the Parameters from the model and
    return them as a tuple as well as their original attribute names.
    The weights must be re-loaded with `load_weights` before the model
    can be used again.
    Note that this function modifies the model in place and after this
    call, mod.parameters() will be empty.
    """
    orig_params = tuple(mod.parameters())
    # Remove all the parameters in the model
    names = []
    for name, p in list(mod.named_parameters()):
        _del_nested_attr(mod, name.split("."))
        names.append(name)

    # Make params regular Tensors instead of nn.Parameter
    params = tuple(p.detach().requires_grad_() for p in orig_params)
    return params, names

def load_weights(mod, names, params):
    """
    Reload a set of weights so that `mod` can be used again to perform a forward pass.
    Note that the `params` are regular Tensors (that can have history) and so are left
    as Tensors. This means that mod.parameters() will still be empty after this call.
    """
    for name, p in zip(names, params):
        _set_nested_attr(mod, name.split("."), p)
        
def calculate_NTK(model,x,device='cpu',MODE='samples'):
    """
    INPUTS:
        model: torch.nn.Module 
        x: torch.Tensor
        device: 'cpu',
        MODE: 'minima'
    
    OUTPUTS:
        NTK: torch.Tensor
    
    Calculates the NTK for a model, p_dict a state dictionary, and x, a single tensor fed into the model
    
    The NTK is the grammian of the Jacobian of the model output to w.r.t. the weights of the model
    
    This function will output the NTK such that the minima matrix size is used. If the Jacobian is an NxM
    matrix, then the NTK is formulated so that if N < M; NTK is NxN. If M<N, then NTK is MxM.
    
    #EXAMPLE USAGE:
    device='cpu'
    model = MODEL() #a torch.nn.Module object 
    model.to(device)
    state_dict = model.state_dict()

    x_test = np.ones((100,1,28,28),dtype=np.float32)
    x_test = torch.from_numpy(x_test)

    NTK = calculate_NTK(model,x_test)
    """
    if not(MODE in ['minima','samples','params']):
        raise ValueError("MODE must be one of 'minima','samples','params'")
    
    x = x.to(device)
    x.requires_grad=False
    N = x.shape[0]
    M = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    #We need to create a clone of the model or else we make it unusable as part of the trickery 
    #to get pytorch to do what we want. Unforutantely, this exlcludes super big models. but, eh.
    model_clone = copy.deepcopy(model)
    
    params, names = extract_weights(model_clone)
    def model_ntk(*args,model=model_clone, names=names):
        params = tuple(args)
        load_weights(model, names, params)
        return model(x)
    
    Js = torch.autograd.functional.jacobian(model_ntk, tuple(params), create_graph=False, vectorize=True)
    
    Js = list(Js)
    #Js = [element for tupl in Js for element in tupl]
    #collapse the tensors
    for i,tensor in enumerate(Js):
        Js[i] = tensor.reshape(N,-1)
    
    J = torch.cat(Js,axis=1)
    
    if MODE=='minima':
        if N < M: #if datasize points is less than number of parameters:
            NTK = torch.matmul(J,J.T)

        if N >= M:#if number of parameters is less than datasize:
            NTK = torch.matmul(J.T,J)
    elif MODE=='samples':
        NTK = torch.matmul(J,J.T)
    elif MODE=='params':
        NTK = torch.matmul(J.T,J)
    
    return NTK

In [3]:
@njit
def zero_pad(A,pad):
    N, F, H, W = A.shape
    P = np.zeros((N, F, H+2*pad, W+2*pad),dtype=np.float32)
    P[:,:,pad:H+pad,pad:W+pad] = A
    return P

In [4]:
@njit
def calc_dw(x,w,b,pad,stride,H_,W_):
    """
    Calculates the derivative of conv(x,w) with respect to w
    
    output is shape:
        [datapoints, in_channels out_filters, kernel_height, kernel_width, out_filters, data_height, data_width
    
    'n f1 f2 c kh kw dh dw -> n (c f1 kh kw) (f2 dh dw)'
    """
    dx, dw, db = None, None, None
    N, C, H, W = x.shape
    F, _, HH, WW = w.shape
    
    #dw = np.zeros((N,F,F,C,HH,WW,H_,W_),dtype=np.float32)
    
    dw = np.zeros((N,C,F,HH,WW,F,H_,W_),dtype=np.float32)

    xp = zero_pad(x,pad)
    #high priority, how to vectorize this operation?
    for n in range(N):
        for f in range(F):
            for i in range(HH): 
                for j in range(WW): 
                    for k in range(H_): 
                        for l in range(W_): 
                            for c in range(C): 
                                dw[n,c,f,i,j,f,k,l] += xp[n, c, i+stride*k, j+stride*l]                             
    
    return dw.reshape((N,(C*F*HH*WW),(F*H_*W_)))

@njit
def calc_dx(x,w,b,pad,stride,H_,W_):
    '''
    calculates the derivative of conv(x,w) with respect to x
    
    output is a nd-array of shape n x ch_in x og_h x og_w x (h_out w_out ch_out)
    '''
    dx, dw, db = None, None, None
    N, C, H, W = x.shape
    F, _, HH, WW = w.shape 

    dx = np.zeros((C,H,W,F,H_,W_,),dtype=np.float32)
    #high priority, how to vectorize this operation? maybe with np.chunk,split?
    for f in range(F): 
        for i in range(H): 
            for j in range(W):
                for k in range(H_): 
                    for l in range(W_):
                        for c in range(C): 
                            if i-stride*k+pad > HH-1 or j-stride*l+pad > WW-1:
                                continue #this is alternative to padding w with zeros.
                            if i-stride*k+pad < 0 or j-stride*l+pad < 0:
                                continue #this is alternative to padding w with zeros.
                            dx[c,i,j,f,k,l] += w[f, c, i-stride*k+pad, j-stride*l+pad]
    #'c ih iw f oh ow -> (c ih iw) (f oh ow)'
    return dx.reshape(((C*H*W),(F*H_*W_)))


In [5]:
#sparsely
def calc_dw_sparsely(x,w,b,pad,stride,H_,W_):
    """
    Calculates the derivative of conv(x,w) with respect to w
    
    output is shape:
        [datapoints, in_channels out_filters, kernel_height, kernel_width, out_filters, data_height, data_width
    
    'n f1 f2 c kh kw dh dw -> n (c f1 kh kw) (f2 dh dw)'
    """
    dx, dw, db = None, None, None
    N, C, H, W = x.shape
    F, _, HH, WW = w.shape
    
    #dw = np.zeros((N,F,F,C,HH,WW,H_,W_),dtype=np.float32)
    
    dw = np.zeros((N,C,F,HH,WW,F,H_,W_),dtype=np.float32)

    xp = zero_pad(x,pad)
    #high priority, how to vectorize this operation?
    for n in range(N):
        for f in range(F):
            for i in range(HH): 
                for j in range(WW): 
                    for k in range(H_): 
                        for l in range(W_): 
                            for c in range(C): 
                                dw[n,c,f,i,j,f,k,l] += xp[n, c, i+stride*k, j+stride*l]                             
    
    return dw.reshape((N,(C*F*HH*WW),(F*H_*W_)))

In [6]:
def relu(X,normalize=False):
    X = F.relu(X)
    if normalize:
        return np.sqrt(2*np.pi/(np.pi-1))*(X-1/np.sqrt(2*np.pi))
    else:
        return X
    

# #Identity
# def activation(x):
#     return x

# @njit
# def d_activation(x):
#     return np.ones(np.shape(x),dtype=np.float32) 


#Tanh
def activation(x):
    return torch.tanh(x)

@njit
def d_activation(x):
    return np.cosh(x)**-2

In [7]:
SEED = 0

In [8]:
def NTK_weights(m):
    if isinstance(m, nn.Linear):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]
    if isinstance(m, nn.Conv2d):
        print(m.weight.shape)
        nn.init.normal_(m.weight.data)#/m.weight.shape[0]
        if m.bias != None:
            nn.init.normal_(m.bias.data)#/m.weight.shape[0]

In [9]:
# Easy NTK expects one output alone
class dumb_small(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small, self).__init__()
        
        self.d1 = torch.nn.Conv2d(1,2,3,bias=True)#28 -> 26

        self.d2 = torch.nn.Conv2d(2,4,3,stride=2,padding=1,bias=True)#26 -> 13
        
        self.d3 = torch.nn.Conv2d(4,8,3,stride=1,padding=0,bias=True)#13 -> 11
        
        self.d4 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#11 -> 9
        
        self.d5 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#9 -> 7

        self.d6 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#7 -> 5

        self.d7 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#5 -> 3
        
        self.d8 = torch.nn.Linear(3*3*8,16,bias=True)
        
        self.d9 = torch.nn.Linear(16,1,bias=True)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0)) / np.sqrt(2*3*3)
        x_2 = activation(self.d2(x_1)) / np.sqrt(4*3*3)
        x_3 = activation(self.d3(x_2)) / np.sqrt(8*3*3)
        x_4 = activation(self.d4(x_3)) / np.sqrt(8*3*3)
        x_5 = activation(self.d5(x_4)) / np.sqrt(8*3*3)
        x_6 = activation(self.d6(x_5)) / np.sqrt(8*3*3)
        x_7 = activation(self.d7(x_6)) / np.sqrt(8*3*3)
        x_8 = x_7.reshape(-1,3*3*8)
        x_9 = activation(self.d8(x_8)) / np.sqrt(16)
        x_10 = self.d9(x_9)
        return x_10 
    
# Easy NTK expects one output alone
class dumb_small_layerwise(torch.nn.Module):
    '''
    simple network for test cases
    
    
    It seems like bias vectors aren't trivially added.
    '''
    def __init__(self,):
        super(dumb_small_layerwise, self).__init__()
        
        self.d1 = torch.nn.Conv2d(1,2,3,bias=True)#28 -> 26

        self.d2 = torch.nn.Conv2d(2,4,3,stride=2,padding=1,bias=True)#26 -> 13
        
        self.d3 = torch.nn.Conv2d(4,8,3,stride=1,padding=0,bias=True)#13 -> 11
        
        self.d4 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#11 -> 9
        
        self.d5 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#9 -> 7

        self.d6 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#7 -> 5

        self.d7 = torch.nn.Conv2d(8,8,3,stride=1,padding=0,bias=True)#5 -> 3
        
        self.d8 = torch.nn.Linear(3*3*8,16,bias=True)
        
        self.d9 = torch.nn.Linear(16,1,bias=True)
        
    def forward(self, x_0):
        x_1 = activation(self.d1(x_0)) / np.sqrt(2*3*3)
        x_2 = activation(self.d2(x_1)) / np.sqrt(4*3*3)
        x_3 = activation(self.d3(x_2)) / np.sqrt(8*3*3)
        x_4 = activation(self.d4(x_3)) / np.sqrt(8*3*3)
        x_5 = activation(self.d5(x_4)) / np.sqrt(8*3*3)
        x_6 = activation(self.d6(x_5)) / np.sqrt(8*3*3)
        x_7 = activation(self.d7(x_6)) / np.sqrt(8*3*3)
        x_8 = x_7.reshape(-1,3*3*8)
        x_9 = activation(self.d8(x_8)) / np.sqrt(16)
        x_10 = self.d9(x_9)
        return x_10, x_9, x_8, x_7, x_6, x_5, x_4, x_3, x_2, x_1, x_0

In [10]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cpu'

model_small = dumb_small()
model_small.apply(NTK_weights)
model_small.to(device)

torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
device='cuda'

model = dumb_small_layerwise()
model.apply(NTK_weights)
model.to(device)

x_test = np.random.normal(0,1,(3,1,28,28)).astype(np.float32) #n c_in, h, w
x_test = torch.from_numpy(x_test)

torch.Size([2, 1, 3, 3])
torch.Size([4, 2, 3, 3])
torch.Size([8, 4, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([16, 72])
torch.Size([1, 16])
torch.Size([2, 1, 3, 3])
torch.Size([4, 2, 3, 3])
torch.Size([8, 4, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([8, 8, 3, 3])
torch.Size([16, 72])
torch.Size([1, 16])


In [11]:
assert(np.allclose(model.d1.weight.detach().cpu().numpy(), model_small.d1.weight.detach().cpu().numpy()))

# Easy_NTK

In [12]:
NTK_easy = calculate_NTK(model_small,x_test).detach().numpy()

# Pytorch Autograd

In [13]:
model_small.zero_grad()
y = model_small(x_test)

In [14]:
#this method agrees between model layerwise and model small; meaning that the calculation is indepdent of those
#two models. the insinuation is somehting is wrong with both my methods for calculating,--- the same thing, since
#they agree with one another.

#in the future we would iterate over layers instead of like this...
layer_components_w1 = [] 
layer_components_w2 = []
layer_components_w3 = []
layer_components_w4 = []
layer_components_w5 = []
layer_components_w6 = []
layer_components_w7 = []
layer_components_w8 = []
layer_components_w9 = []

layer_components_b1 = []
layer_components_b2 = []
layer_components_b3 = []
layer_components_b4 = []
layer_components_b5 = []
layer_components_b6 = []
layer_components_b7 = []
layer_components_b8 = []
layer_components_b9 = []

for output in y:
    model_small.zero_grad()
    
    output.backward(retain_graph=True)

    #Get the tensors
    w1_grad = model_small.d1.weight.grad.detach().numpy()
    w2_grad = model_small.d2.weight.grad.detach().numpy()
    w3_grad = model_small.d3.weight.grad.detach().numpy()
    w4_grad = model_small.d4.weight.grad.detach().numpy()
    w5_grad = model_small.d5.weight.grad.detach().numpy()
    w6_grad = model_small.d6.weight.grad.detach().numpy()
    w7_grad = model_small.d7.weight.grad.detach().numpy()
    w8_grad = model_small.d8.weight.grad.detach().numpy()
    w9_grad = model_small.d9.weight.grad.detach().numpy()
    
    b1_grad = model_small.d1.bias.grad.detach().numpy()
    b2_grad = model_small.d2.bias.grad.detach().numpy()
    b3_grad = model_small.d3.bias.grad.detach().numpy()
    b4_grad = model_small.d4.bias.grad.detach().numpy()
    b5_grad = model_small.d5.bias.grad.detach().numpy()
    b6_grad = model_small.d6.bias.grad.detach().numpy()
    b7_grad = model_small.d7.bias.grad.detach().numpy()
    b8_grad = model_small.d8.bias.grad.detach().numpy()
    b9_grad = model_small.d9.bias.grad.detach().numpy()

    #reshape and append. deep copy neccessary or else they are the same objects
    layer_components_w1.append(w1_grad.reshape(-1).copy())
    layer_components_w2.append(w2_grad.reshape(-1).copy())
    layer_components_w3.append(w3_grad.reshape(-1).copy())
    layer_components_w4.append(w4_grad.reshape(-1).copy())
    layer_components_w5.append(w5_grad.reshape(-1).copy())
    layer_components_w6.append(w6_grad.reshape(-1).copy())
    layer_components_w7.append(w7_grad.reshape(-1).copy())
    layer_components_w8.append(w8_grad.reshape(-1).copy())
    layer_components_w9.append(w9_grad.reshape(-1).copy())
    
    layer_components_b1.append(b1_grad.reshape(-1).copy())
    layer_components_b2.append(b2_grad.reshape(-1).copy())
    layer_components_b3.append(b3_grad.reshape(-1).copy())
    layer_components_b4.append(b4_grad.reshape(-1).copy())
    layer_components_b5.append(b5_grad.reshape(-1).copy())
    layer_components_b6.append(b6_grad.reshape(-1).copy())
    layer_components_b7.append(b7_grad.reshape(-1).copy())
    layer_components_b8.append(b8_grad.reshape(-1).copy())
    layer_components_b9.append(b9_grad.reshape(-1).copy())

In [15]:
layer_components_w1 = np.array(layer_components_w1)
layer_components_w2 = np.array(layer_components_w2)
layer_components_w3 = np.array(layer_components_w3)
layer_components_w4 = np.array(layer_components_w4)
layer_components_w5 = np.array(layer_components_w5)
layer_components_w6 = np.array(layer_components_w6)
layer_components_w7 = np.array(layer_components_w7)
layer_components_w8 = np.array(layer_components_w8)
layer_components_w9 = np.array(layer_components_w9)

layer_components_b1 = np.array(layer_components_b1)
layer_components_b2 = np.array(layer_components_b2)
layer_components_b3 = np.array(layer_components_b3)
layer_components_b4 = np.array(layer_components_b4)
layer_components_b5 = np.array(layer_components_b5)
layer_components_b6 = np.array(layer_components_b6)
layer_components_b7 = np.array(layer_components_b7)
layer_components_b8 = np.array(layer_components_b8)
layer_components_b9 = np.array(layer_components_b9)

In [16]:
autograd_NTK = layer_components_w1 @ layer_components_w1.T+\
    layer_components_w2 @ layer_components_w2.T+\
    layer_components_w3 @ layer_components_w3.T+\
    layer_components_w4 @ layer_components_w4.T+\
    layer_components_w5 @ layer_components_w5.T+\
    layer_components_w6 @ layer_components_w6.T+\
    layer_components_w7 @ layer_components_w7.T+\
    layer_components_w8 @ layer_components_w8.T+\
    layer_components_w9 @ layer_components_w9.T+\
    layer_components_b1 @ layer_components_b1.T+\
    layer_components_b2 @ layer_components_b2.T+\
    layer_components_b3 @ layer_components_b3.T+\
    layer_components_b4 @ layer_components_b4.T+\
    layer_components_b5 @ layer_components_b5.T+\
    layer_components_b6 @ layer_components_b6.T+\
    layer_components_b7 @ layer_components_b7.T+\
    layer_components_b8 @ layer_components_b8.T+\
    layer_components_b9 @ layer_components_b9.T

# Layerwise

In [17]:
x_test = x_test.to('cuda')
x_10, x_9, x_8, x_7, x_6, x_5, x_4, x_3, x_2, x_1, x_0 = model(x_test)

#These need to be numpy
Ws = []
Ws.append(torch.tensor([0.0],dtype=torch.float32)) #spacer
Ws.append(torch.tensor([0.0],dtype=torch.float32)) #spacer 
Ws.append(torch.tensor([0.0],dtype=torch.float32))
Ws.append(torch.tensor([0.0],dtype=torch.float32))
Ws.append(torch.tensor([0.0],dtype=torch.float32)) 
Ws.append(torch.tensor([0.0],dtype=torch.float32)) #spacer reshape is a layer
Ws.append(torch.tensor([0.0],dtype=torch.float32)) #spacer reshape is a layer
Ws.append(torch.tensor([0.0],dtype=torch.float32)) #spacer reshape is a layer
Ws.append(model.d8.weight.detach())
Ws.append(model.d9.weight.detach())


#Kernel Matrices, Need to be numpy
Ks = []
Ks.append(model.d1.weight.detach())
Ks.append(model.d2.weight.detach())
Ks.append(model.d3.weight.detach())
Ks.append(model.d4.weight.detach())
Ks.append(model.d5.weight.detach())
Ks.append(model.d6.weight.detach())
Ks.append(model.d7.weight.detach())
Ks.append(torch.tensor([0.0],dtype=torch.float32)) #spacer
Ks.append(torch.tensor([0.0],dtype=torch.float32)) #spacer
Ks.append(torch.tensor([0.0],dtype=torch.float32)) #spacer


Xs = [] # Xs are shape (output x #DP) ; however, typical python notation is reversed, so we take transpose here
Xs.append(x_0.T.detach())
Xs.append(x_1.T.detach())
Xs.append(x_2.T.detach())
Xs.append(x_3.T.detach())
Xs.append(x_4.T.detach())
Xs.append(x_5.T.detach())
Xs.append(x_6.T.detach())
Xs.append(x_7.T.detach())
Xs.append(x_8.T.detach())
Xs.append(x_9.T.detach())

#This is used to create arrays-- needs to be integer list to play nice with compilers
ds_int = []
ds_int.append(0)
ds_int.append(2*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(4*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(8*3*3) #channels_out * kernel_height * kernel_width
ds_int.append(16) #number output features
ds_int.append(1) #number output features

ds_array = [] #this is for the NTK formulation, 
#ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device)) #first element is a spacer, could be anything.
ds_array.append(torch.tensor([2*3*3],dtype=torch.float32).to(device)) #The rest, even if you dont use NTK formulation, would be 1
ds_array.append(torch.tensor([4*3*3],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([8*3*3],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([8*3*3],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([8*3*3],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([8*3*3],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([8*3*3],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([0.0],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([16.0],dtype=torch.float32).to(device))
ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device)) #first element is a spacer, could be anything.
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device)) #The rest, even if you dont use NTK formulation, would be 1
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))
# ds_array.append(torch.tensor([1.0],dtype=torch.float32).to(device))

padding = []
padding.append(0)
padding.append(1)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)
padding.append(0)

strides = []
strides.append(1)
strides.append(2)
strides.append(1)
strides.append(1)
strides.append(1)
strides.append(1)
strides.append(1)
strides.append(0)
strides.append(0)
strides.append(0)

layers =[model.d1,
        model.d2,
        model.d3,
        model.d4,
        model.d5,
        model.d6,
        model.d7,
        0.0,
        model.d8,
        model.d9]

In [18]:
len(ds_array)

10

# Improve this algorithm's speed: here is the original

In [19]:
@njit
def cross(X):
    return X.T.dot(X)

def cross_pt_nonp(X,device='cuda'):
    X = X.to(device)
    return X.T.matmul(X)

def cross_pt(X,device='cuda'):
    X = torch.from_numpy(X).to(device)
    X = X.to(device)
    return X.T.matmul(X).cpu()

def d_activationt(x):
    return torch.cosh(x)**-2

In [20]:
#here are the updates we plan:
def compute_NTK_CNN(Ws, Ks, Xs, d_int, d_array, strides, padding, layers, device="cpu"):
    components = []
    
    L = len(Xs)-1 #number of layers, Xs goes from inputs to right before outputs; X_0 is the input, X_L CK
    n = Xs[0].shape[-1] #number of datapoints

    #holds the derivatives of activation, first value is empty list...?; just a spacer, replace with array
    Ds_dense = [np.array([[0.0]],dtype=np.float32)] 
    Ds_conv = [np.array([[0.0]],dtype=np.float32)]
    s_matrices = []
    with torch.no_grad():
        ####################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Linear):
                Ds_dense.append(d_activationt(layers[l](Xs[l].T)).T)
            else:
                Ds_dense.append(np.array([[0.0]],dtype=np.float32))
        ################################################################################################
        for l in range(0,L):
            if isinstance(layers[l],torch.nn.Conv2d):
                Ds_conv.append(d_activationt(layers[l](Xs[l].T)).reshape(n,-1).T)
            else:
                Ds_conv.append(np.array([[0.0]],dtype=np.float32))      
        ####################################################################################################
        S = torch.tensor([1.0],dtype=torch.float32).to(device) #this models the backward propogation:   
        for l in range(L,-1,-1):
            if isinstance(layers[l], torch.nn.Linear):

                components.append(cross_pt_nonp(S,device)*cross_pt_nonp(Xs[l],device)/d_array[l])

                W = torch.ones((d_int[l],n),dtype=torch.float32).to(device) * S
                components.append(cross_pt_nonp(W,device).to(device)/d_array[l])

            elif isinstance(layers[l], torch.nn.Conv2d):
                if len(S.shape) == 2: #this should only affect the very last layer, at which point, who cares.
                    S = S[None,:,:]
                    
                dw = calc_dw(x=Xs[l].T.cpu().numpy(),w=Ks[l].cpu().numpy(),b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                
                W = torch.matmul(torch.from_numpy(dw).to(device),S.to(device))
                
                #We should bring this to zhichao or sombody and ask if there is obviously something faster?
                #W = np.diagonal(W,0,2,0)
                W = torch.diagonal(W,0,0,2)

                components.append(cross_pt_nonp(W,device).to(device)/d_array[l])

                N = Ks[l].shape[0]
                W = np.split(S.cpu().numpy(),N,axis=1)
                #W = torch.split(S,N,dim=1)
                
                W = np.array(W)
                #W = torch.stack(W)
                
                W = np.sum(W,axis=(1,2))
                #W = torch.sum(W,dim=(1,2))
                
                components.append(torch.from_numpy(cross(W,)).to(device)/d_array[l])
                #components.append(cross_pt_nonp(W,device))

            #############################
            #now we setup S for the next loop by treating appropriately
            if l==0:
                break

            if isinstance(layers[l], torch.nn.Linear):
                S = torch.matmul(S.T,Ws[l]) / torch.sqrt(d_array[l])
                if len(S.shape) < 2:
                    S = S[:,None] #expand dimension along axis 1
                if not(isinstance(layers[l-1],float)): #this exludes the reshaping layer
                    S = Ds_dense[l]*S
                else: #and when the reshaping layer occurs we need to apply this instead
                    S = Ds_conv[l-1]*S.T

            elif isinstance(layers[l], torch.nn.Conv2d):
                dx = calc_dx(x=Xs[l].T.cpu().numpy(),w=Ks[l].cpu().numpy(),b=0,pad=padding[l],stride=strides[l],H_=Xs[l+1].shape[1],W_=Xs[l+1].shape[0])
                S = (torch.from_numpy(dx[None,:,:]).to(device) @ S) / torch.sqrt(d_array[l])
                S = Ds_conv[l]*S
            
    return components#, dws, s_matrices

In [21]:
#%%timeit
components = compute_NTK_CNN(Ws, Ks, Xs, ds_int, ds_array, strides, padding, layers, device='cuda')

In [22]:
NTK_layerwise = torch.sum(torch.stack(components),dim=0).cpu().numpy()

In [23]:
sum(p.numel() for p in model.parameters() if p.requires_grad)
#lol yikes

3913

# Compare Against One another

In [24]:
NTK_layerwise

array([[4.290627 , 4.4077263, 4.3809752],
       [4.4077263, 4.5796604, 4.5244513],
       [4.3809752, 4.5244513, 4.5322475]], dtype=float32)

In [25]:
NTK_easy

array([[4.2906275, 4.407726 , 4.3809757],
       [4.4077253, 4.5796614, 4.524452 ],
       [4.3809757, 4.524452 , 4.532248 ]], dtype=float32)

In [26]:
autograd_NTK

array([[4.290628 , 4.4077272, 4.3809757],
       [4.4077272, 4.5796614, 4.524452 ],
       [4.3809757, 4.524452 , 4.5322485]], dtype=float32)

In [27]:
print(np.allclose(NTK_layerwise,NTK_easy))
print(np.allclose(NTK_layerwise,autograd_NTK,rtol=1e-1))
print(np.allclose(NTK_easy,autograd_NTK))

True
True
True


# Compare Layerwise Components

In [28]:
print(layer_components_w1 @ layer_components_w1.T)
print(layer_components_w2 @ layer_components_w2.T)
print(layer_components_w3 @ layer_components_w3.T)
print(layer_components_w4 @ layer_components_w4.T)
print(layer_components_w5 @ layer_components_w5.T)
print(layer_components_w6 @ layer_components_w6.T)
print(layer_components_w7 @ layer_components_w7.T)
print(layer_components_w8 @ layer_components_w8.T)
print(layer_components_w9 @ layer_components_w9.T)
print(' ')
print(layer_components_b1 @ layer_components_b1.T)
print(layer_components_b2 @ layer_components_b2.T)
print(layer_components_b3 @ layer_components_b3.T)
print(layer_components_b4 @ layer_components_b4.T)
print(layer_components_b5 @ layer_components_b5.T)
print(layer_components_b6 @ layer_components_b6.T)
print(layer_components_b7 @ layer_components_b7.T)
print(layer_components_b8 @ layer_components_b8.T)
print(layer_components_b9 @ layer_components_b9.T)

NTK_components_autograd = [
    layer_components_w9 @ layer_components_w9.T,
    layer_components_b9 @ layer_components_b9.T,
    layer_components_w8 @ layer_components_w8.T,
    layer_components_b8 @ layer_components_b8.T,
    layer_components_w7 @ layer_components_w7.T,
    layer_components_b7 @ layer_components_b7.T,
    layer_components_w6 @ layer_components_w6.T,
    layer_components_b6 @ layer_components_b6.T,
    layer_components_w5 @ layer_components_w5.T,
    layer_components_b5 @ layer_components_b5.T,
    layer_components_w4 @ layer_components_w4.T,
    layer_components_b4 @ layer_components_b4.T,
    layer_components_w3 @ layer_components_w3.T,
    layer_components_b3 @ layer_components_b3.T,
    layer_components_w2 @ layer_components_w2.T,
    layer_components_b2 @ layer_components_b2.T,
    layer_components_w1 @ layer_components_w1.T,
    layer_components_b1 @ layer_components_b1.T,
]

[[ 0.00214623 -0.00179957 -0.00051422]
 [-0.00179957  0.00404241 -0.0011052 ]
 [-0.00051422 -0.0011052   0.01060728]]
[[ 0.00111821 -0.00055514  0.00021577]
 [-0.00055514  0.00220726  0.00043866]
 [ 0.00021577  0.00043866  0.00231693]]
[[0.02044179 0.02321871 0.02447354]
 [0.02321871 0.03179909 0.02974743]
 [0.02447354 0.02974743 0.03389359]]
[[0.03434303 0.03549245 0.03707688]
 [0.03549245 0.04039492 0.0401036 ]
 [0.03707688 0.0401036  0.04381075]]
[[0.09600651 0.10147955 0.09412385]
 [0.10147955 0.11025555 0.100274  ]
 [0.09412385 0.100274   0.09491354]]
[[0.21020919 0.22095677 0.21675074]
 [0.22095677 0.2357184  0.22866714]
 [0.21675074 0.22866714 0.2261468 ]]
[[0.27614293 0.28773996 0.28657293]
 [0.28773996 0.30187675 0.29930374]
 [0.28657293 0.29930374 0.29934138]]
[[0.14711633 0.14732318 0.14695993]
 [0.14732318 0.15017757 0.1481148 ]
 [0.14695993 0.1481148  0.14941087]]
[[0.40693706 0.40290454 0.40761644]
 [0.40290454 0.40157023 0.40588877]
 [0.40761644 0.40588877 0.41382027]]
 

In [29]:
l = 9

print(np.allclose(NTK_components_autograd[l],components[l].cpu().numpy(),1e-1))
print(NTK_components_autograd[l])
print(components[l])

True
[[0.3212226  0.34508857 0.32697394]
 [0.34508857 0.37103772 0.35194153]
 [0.32697394 0.35194153 0.33442992]]
tensor([[0.3212, 0.3451, 0.3270],
        [0.3451, 0.3710, 0.3519],
        [0.3270, 0.3519, 0.3344]], device='cuda:0')
